# Portfolio Optimization
The portfolio optimization module is a collection of functions that can be used to optimize a portfolio of assets. The module contains functions to calculate the efficient frontier, the optimal portfolio, the return of the optimal portfolio and more.

Prototyping and experimenting with this module should be easy and fun. The documentation is therefore written in the form of a tutorial with examples.

### Modules
The portfolio optimization module consists of the following modules:
- **data_processing**: This module contains functions to process data. For example, it contains functions to calculate the expected returns and the covariance matrix of a portfolio.
- **data_collection**: This module contains functions to collect data. For example, it contains functions to collect price data from CoinMetrics.
- **optimization**: This module contains functions to optimize a portfolio. For example, it contains functions to calculate the efficient frontier and the optimal portfolio.
- **portfolio**: This module contains objects to represent a portfolio. This is mainly used to store and rebalance a portfolio.
- **backtesting**: This module contains functions to backtest a portfolio.

In [1]:
# Importing data processing modules, such as expected returns.
from portfolio_optimization.data_processing import *
# Importing data collection modules, such as getting assets.
from portfolio_optimization.data_collection import *
# Importing optimization modules, such as the optimizer. This is the main module for generating portfolios.
from portfolio_optimization.optimization import *
# Importing portfolio modules, such as the portfolio class. This is the main module for using optimizers
from portfolio_optimization.portfolio import *
# Importing backtesting modules, such as the backtester. This is the main module for backtesting portfolios.
from portfolio_optimization.backtesting import *


## Note: The following imports are for the documentation only
from tokens.get_assets import *
import numpy as np
import pandas as pd
from datetime import timedelta
from dateutil.relativedelta import relativedelta

## Data Collection
The first step is to collect data. The data collection module contains functions to collect data from CoinMetrics. The data is fetched as a git repository and stored in a local directory. The data is then read of the CSV files in this directory (in our case the `data` directory). The raw data is converted to a Pandas DataFrame returned by `get_historical_prices_for_assets`. Since the data is containing a lot of information, we will only use the `ReferenceRate` and `CapMrktEstUSD` columns. The `ReferenceRate` column contains the price of the asset and the `CapMrktEstUSD` column contains the market cap of the asset. The market cap is used to calculate the weights of the assets in the black litterman portfolio.

In [2]:
# Choose the asset class and the assets
asset_list = get_tickers()
asset_class = "high_risk_tickers"

_df = get_historical_prices_for_assets(
    asset_list[asset_class],
    time_range=timedelta(days=365 * 1 + 120),  # 1 years
    interested_columns=["ReferenceRate", "CapMrktEstUSD"],
)

High risk tickers: 12
Medium risk tickers: 60
Low risk tickers: 8


/Users/arguiot/Developer/Pyratz/portfolio/portfolio_optimization/data_collection/get_crypto_price_range.py:41: DtypeWarning: Columns (146) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


## Data Processing
The data processing module is an internal module that is not going to be used by the frontend (Manager), here this is just some quick data cleaning for the notebook.

In [3]:
# Filter out all columns containing `_` in their name
df = _df.loc[:, ~_df.columns.str.contains("_")]

# Get all the market caps
mcaps = _df.loc[:, _df.columns.str.contains("CapMrktEstUSD")]
mcaps.columns = mcaps.columns.str.replace("_CapMrktEstUSD", "")
mcaps.replace(np.nan, 0, inplace=True)

start_date_portfolio = df.index[0] + relativedelta(days=120)

/var/folders/d2/1qh2h9ys19z3375prb_0cb8w0000gn/T/ipykernel_33529/3818043710.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mcaps.replace(np.nan, 0, inplace=True)


## Delegates & Data pipelines
`Portfolio` and `Optimization` modules were designed with pipeline architecture in mind. That means that some steps in each process can be delegated to external logic.

### Optimization
Delegating for an `GeneralOptimization` class is easy. You just need to extend `GeneralOptimizationDelegate`:

In [4]:
class CustomMarkowitzDelegate(GeneralOptimizationDelegate):
    def setup(self, optimization_object: Markowitz):
        print("Setting up the Markowitz object")
        optimization_object.mode = optimization_object.CovMode.LEDOIT_WOLF # See CovMode on the Markowitz class for more options
        optimization_object.efficient_portfolio = (
            optimization_object.EfficientPortfolio.MAX_SHARPE # See EfficientPortfolio on the Markowitz class for more options
        )

        # Now we need to set the expected returns and the covariance matrix
        if optimization_object.cov_matrix is None:
            optimization_object.cov_matrix = optimization_object.get_cov_matrix()

        optimization_object.rets = expected_returns.mean_historical_return(
            optimization_object.df
        )

        return super().setup(optimization_object) # Setup can be cascaded

### Portfolio
Delegating for an `Portfolio` class is super powerful for custom rebalancing logic. You just need to extend `PortfolioDelegate`:

In [5]:
class OptRebalancingPortfolioDelegate(PortfolioDelegate):
    def rebalance(
        self, holdings: pd.Series, prices: pd.Series, target_weights: pd.Series
    ) -> pd.Series:
        diff = optimize_trades(
            holdings=holdings,
            new_target_weights=target_weights,
            prices=prices,
            min_W=0.0,
            max_W=1.0,
            external_movement=0,
        )

        new_holdings = pd.Series(diff, index=holdings.index) + holdings
        return new_holdings

## Setting up the backtesting parameters
Backtesting plays a central role in this package. Various parameters can be set to customize the backtesting process. The following parameters can be set:

In [6]:
initial_bid = 1000 # We start with 1000 USD

max_weight = {"*": 1.0} # Can be: {'*': 1.0} or {'*': 0.5, 'BTC': 0.15}

min_weight = { '*': 0.0} # Can be: {'*': 0.0} or {'*': 0.5, 'BTC': 0.05}

budget = {} # Can be: {'uni': 1.2} for extra 20% for UNI

yield_data = pd.Series()
for asset in asset_list[asset_class]:
    yield_data[asset] = 0.075

weight_threshold = 0.01

lambda_var = 0.1
lambda_u = 0.1

rebalance_frequency = "1W"
adjust_holdings = True # Wether to adjust the holdings daily to obtain the target weights

## Portfolio Construction & backtesting
Constructing a portfolio is done by using the `Portfolio` class. The `Portfolio` class is initialized with the following parameters:
- **base_value**: The base value of the portfolio. This is the value of the portfolio at the start of the backtesting period.
- **initial_prices**: The initial prices of the assets in the portfolio. This is used to calculate the initial weights of the assets in the portfolio.
- **optimiser**: The chosen optimiser. Make sure to `bind` the delegate to the optimiser.
- **max_weight**: The maximum weight of an asset in the portfolio. This is used to set the bounds of the optimisation problem.
- **min_weight**: The minimum weight of an asset in the portfolio. This is used to set the bounds of the optimisation problem.
- **weight_threshold**: The threshold in dollars for the weights of the assets in the portfolio. This will be passed to the rebalancing delegate
- **budget**: The budget of the portfolio. This is used with `RiskParity`
- **lambda_var**: The variance risk budgeting lambda parameter. This is used with `RiskParity`
- **lambda_mean**: The mean risk budgeting lambda parameter. This is used with `RiskParity`


The `Backtesting` object can be constructed as follows, where all the `Portfolio` parameters are passed as a dictionary.

In [7]:
portfolio_markowitz = Portfolio(
    base_value=initial_bid,
    initial_prices=df.loc[:start_date_portfolio],
    optimiser=Markowitz.bind(CustomMarkowitzDelegate()),
    max_weight=max_weight,
    min_weight=min_weight,
    weight_threshold=weight_threshold,
    budget=budget,
    lambda_var=lambda_var,
    lambda_u=lambda_u,
)

portfolio_markowitz.delegate = OptRebalancingPortfolioDelegate() # We can set a delegate to the portfolio

backtest = Backtest(
    portfolios={
        "Markowitz": portfolio_markowitz,
    },
    start_date=start_date_portfolio,
    end_date=df.index[-1],
    rebalance_frequency=rebalance_frequency,
    adjust_holdings=adjust_holdings,
    data=df,
    mcaps=mcaps,
    asset_class=asset_class,
)

Setting up the Markowitz object


#### Running the backtest and Saving the data
Running the backtest is done by calling the `run_backtest` method. This method takes the following parameters:
- **look_back_period**: A number of "unit" to look back at every rebalancing
- **look_back_unit**: The unit of the look back period. This can be `D` for days, `W` for weeks, `M` for months or `Y` for years.
- **yield_data**: A `pd.Series` containing the yield of the portfolio. This is APR.

In [9]:
perfs = backtest.run_backtest(
    look_back_period=120,
    look_back_unit="D",
    yield_data=yield_data,
)

# Check if the directory exists, if not, create it
if not os.path.exists(f"./out/{rebalance_frequency}_parity/"):
    os.makedirs(f"./out/{rebalance_frequency}_parity/")

backtest.export_results(
    perfs, f"./out/{rebalance_frequency}_parity/", f"backtest_results_{asset_class}.xlsx"
)

Setting up the Markowitz object
[ 1.38134802e-05  8.59674592e-11  1.68255202e-09  3.36747438e-07
 -7.12491903e-06  6.67805492e-08  1.49087435e-09  1.43099539e-09
  2.01429890e-08 -7.13232183e-06  6.86003976e-09  3.84002566e-09]
optimal_inaccurate
Setting up the Markowitz object
[ 1.92808785e-05  1.32221264e-10  2.39621418e-09  5.38972431e-07
 -9.97537671e-06  9.68440294e-08  2.22837645e-09  2.09267493e-09
  2.69856408e-08 -9.99788330e-06  1.03813651e-08  5.77764468e-09]
optimal_inaccurate
Setting up the Markowitz object
[ 1.53032804e-05  1.03656909e-10  2.10575531e-09  4.17625036e-07
 -7.94021430e-06  8.58435344e-08  1.86427974e-09  1.70535855e-09
  2.34461891e-08 -7.91463850e-06  8.94266864e-09  4.76731841e-09]
optimal_inaccurate
Setting up the Markowitz object
[ 1.11065014e-05  7.61461505e-11  1.40328916e-09  2.96050185e-07
 -5.72076659e-06 -5.70900772e-06  1.24589563e-09  1.29631972e-09
  1.29980795e-08  2.10383251e-10  3.28310088e-09  2.93922869e-09]
optimal_inaccurate
Setting up t

/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/cvxpy/problems/problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/cvxpy/problems/problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_perfo

[-6.27950398e-12 -1.19626195e-16 -2.72656736e-15 -4.59956961e-13
 -4.88999961e-05  4.89000092e-05 -2.06482330e-15 -2.02226123e-15
 -2.35134534e-14 -3.22991386e-16 -5.40550604e-15 -4.86361132e-15]
optimal
Setting up the Markowitz object
[-1.23529164e-11 -2.33332276e-16 -5.29471235e-15 -9.07074822e-13
 -7.84279850e-05  7.84280107e-05 -4.45305268e-15 -4.02034054e-15
 -4.59064730e-14 -6.59864265e-16 -9.72190661e-15 -9.82486145e-15]
optimal
Setting up the Markowitz object
[ 9.88630620e-06  6.11861732e-11  1.34186604e-09  2.60862840e-07
 -5.09442308e-06 -5.07724853e-06  1.19033957e-09  1.03222572e-09
  1.20441678e-08  1.77930420e-10  2.66011420e-09  2.64071935e-09]
optimal
Setting up the Markowitz object
[ 1.69906089e-07  6.90753356e-10  1.56025316e-08  9.75115647e-06
  1.28720438e-07  6.89363352e-07  1.39669461e-08  1.21874081e-08
  1.46054489e-07 -1.03440997e-06  3.24053955e-08  3.09950557e-08]
optimal
Setting up the Markowitz object
[-1.83540128e-05 -9.01507710e-17 -2.05671326e-15 -4.2440

/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_fre

[ 1.45095047e-12 -4.70148458e-22 -9.88891860e-21 -2.14584766e-18
 -5.62253571e-21 -1.45083945e-12 -9.19306410e-21 -7.92006818e-21
 -9.50100802e-20 -1.36901704e-21 -2.22221440e-20 -1.99197838e-20]
optimal
Setting up the Markowitz object
[-9.07260630e-05 -2.36876357e-08 -4.51440979e-07  9.09907147e-06
 -2.49826267e-07 -9.15936998e-05  2.42987385e-05 -3.97387122e-07
 -4.53200099e-06  1.58648593e-04 -1.55575139e-06 -1.09817375e-06]
optimal
Setting up the Markowitz object
[ 1.88737914e-15 -1.87697294e-24 -3.50435048e-23 -1.80411242e-15
 -1.91468736e-23 -1.52427893e-21 -3.40209282e-23 -3.18328962e-23
 -4.31089012e-22 -5.04935879e-24 -1.25663388e-22 -8.62483449e-23]
optimal
Setting up the Markowitz object
[-8.41712553e-06  6.64009239e-11  1.42935805e-09  9.79987441e-06
  7.46881355e-10  3.46307104e-07  1.29986095e-09  1.24788932e-09
  3.88227804e-08 -1.78793583e-06  6.16927067e-09  5.06877850e-09]
optimal
Setting up the Markowitz object
[-1.58530004e-04 -3.84457543e-08 -6.91132787e-07  5.2111

...

/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_fre

[ 2.40804194e-06 -3.05126044e-18 -7.66653047e-17  2.41945670e-06
 -1.47267576e-17 -2.36507737e-15 -8.04832470e-06 -5.12162777e-17
 -7.27582038e-16 -7.00995139e-18  3.22082635e-06 -1.29210845e-16]
optimal
Setting up the Markowitz object
[-9.13731397e-05  2.02727413e-04  3.85145071e-17 -9.13788239e-05
  7.96723970e-18  1.29556897e-15 -1.35232846e-04  2.64981798e-17
  3.43171142e-16  1.96130804e-04 -9.08734073e-05  6.62799049e-17]
optimal
Setting up the Markowitz object
[-3.35337002e-05 -3.83482414e-09 -9.69751013e-08  8.70296460e-06
 -1.91405989e-08 -2.41485826e-06  2.91096851e-05 -6.58202179e-08
 -8.06513437e-07 -8.95262316e-09 -3.18024868e-07 -1.60454420e-07]
optimal
Setting up the Markowitz object
[-1.53193826e-04  5.43517699e-05  6.06307451e-07 -1.57211838e-04
  1.85929889e-04  6.60957662e-07  7.50793039e-05  1.08258309e-04
 -4.64204630e-07  1.05480711e-04 -2.07964094e-04 -1.27216643e-06]
optimal
Setting up the Markowitz object
[ 4.88498131e-15 -2.54922550e-24 -5.86327814e-23 -4.6074

/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_fre

[-9.85131684e-05 -1.34780447e-08  1.18171776e-04 -3.57824307e-06
 -6.26919896e-08 -9.32114189e-06 -2.33114067e-07 -2.19844444e-07
 -3.22734888e-06 -2.84942881e-08 -1.04458218e-06 -4.82751054e-07]
optimal
Setting up the Markowitz object
[-6.20140947e-06  3.27460343e-12  2.93020295e-06  9.88514729e-06
  2.42891023e-11  9.86095680e-08  6.21779307e-11  5.42169800e-11
 -6.71432218e-06  9.13695705e-12  1.07710519e-09  2.23846801e-10]
optimal
Setting up the Markowitz object
[-3.48508737e-06  4.58421682e-10 -3.55494086e-06  9.92832127e-06
  1.97998976e-09  4.05082986e-07  7.31375299e-09  6.78135578e-09
  1.15914243e-07  9.91151265e-10 -3.49089074e-06  1.73962958e-08]
optimal
Setting up the Markowitz object
[ 9.87032239e-06  2.29668677e-11 -3.35291300e-06  1.51176048e-07
  1.03154351e-10  1.93166290e-08  3.79856165e-10  3.47207395e-10
 -3.34511383e-06  4.81023986e-11 -3.34703755e-06  8.50527930e-10]
optimal


/Users/arguiot/Developer/Pyratz/portfolio/env/lib/python3.10/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(
/Users/arguiot/Developer/Pyratz/portfolio/portfolio_optimization/backtesting/Backtesting.py:313: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value["Daily Return"] = daily_return
